In [ ]:
import sys
!{sys.executable} -m pip install nltk

In [37]:
from collections import defaultdict
#{old tag, new tag, condition, variable}

# Since the Penn Treebank data is stored as a list of tuples, we'll create a helper function to get around tuples being immutable objects
def tuple_updater(sentence,idx,new_value):
    sentence[idx] = (sentence[idx][0], new_value)
    return sentence


# Takes in the current rule, the current sentence, and the index of the current word being examined
def rule_parser(rule, sentence, idx):

  rule = (rule[0],rule[1],rule[2],rule[3].split()) # Splits apart the list of paramaters that's stored as a string

  try:
    if rule[0] == sentence[idx][1]:
      # If the rule is of rule template 1...
      if rule[2] == "P1T": # Previous (1 Back) is tagged
        # Using the condition and parameter of rule template 1, examine the current sentence
        if sentence[idx-1][1] == rule[3][0] and idx > 0:
          # If the condition is met update the sentence
          sentence = tuple_updater(sentence,idx,rule[1])
      elif rule[2] == "F1T": # Following (1 Forward) is tagged
        if sentence[idx+1][1] == rule[3][0]:
          sentence = tuple_updater(sentence,idx,rule[1])
      elif rule[2] == "P2T": # Previous (2 Back) is tagged
        if sentence[idx-2][1] == rule[3][0] and idx > 1:
          sentence = tuple_updater(sentence,idx,rule[1])
      elif rule[2] == "F2T": # Following (2 Forward) is tagged
        if sentence[idx+2][1] == rule[3][0]:
          sentence = tuple_updater(sentence,idx,rule[1])
      elif rule[2] == "1P2T": # 1 out of Preceeding 2 is tagged
        if sentence[idx-1][1] == rule[3][0] or sentence[idx-2][1] == rule[3][0]:
          sentence = tuple_updater(sentence,idx,rule[1])
      elif rule[2] == "1F2T": # 1 out of Following 2 is tagged
        if sentence[idx+1][1] == rule[3][0] or sentence[idx+2][1] == rule[3][0]:
          ssentence = tuple_updater(sentence,idx,rule[1])
      elif rule[2] == "1P3T": # 1 out of Preceeding 3 is tagged
        if sentence[idx-1][1] == rule[3][0] or sentence[idx-2][1] == rule[3][0] or sentence[idx-3][1] == rule[3][0]:
          sentence = tuple_updater(sentence,idx,rule[1])
      elif rule[2] == "1F3T": # 1 out of Following 3 is tagged
        if sentence[idx+1][1] == rule[3][0] or sentence[idx+2][1] == rule[3][0] or sentence[idx+3][1] == rule[3][0]:
          sentence = tuple_updater(sentence,idx,rule[1])
      elif rule[2] == "P1TF1T": # Previous is tagged and Following is tagged
        if sentence[idx-1][1] == rule[3][0] and sentence[idx+1][1] == rule[3][1] and idx > 0 and idx < len(sentence)-1:
          sentence = tuple_updater(sentence,idx,rule[1])
      elif rule[2] == "P1TP1T": # Previous is tagged and Previous (2 Back) is tagged
        if sentence[idx-1][1] == rule[3][0] and sentence[idx-2][1] == rule[3][1] and idx > 1:
          sentence = tuple_updater(sentence,idx,rule[1])
      elif rule[2] == "F1TF1T": # Following is tagged and Following (2 Forward) is tagged
        if sentence[idx+1][1] == rule[3][0] and sentence[idx+2][1] == rule[3][1] and idx < len(sentence)-2:
          sentence = tuple_updater(sentence,idx,rule[1])

    return sentence
  except:
    return sentence

def train(master_sentence,answers):
  # Initialize some errors so we can loop while the error isn't improving
  last_accuracy = 1.05
  current_accuracy = 1

  # Initalize the rule book
  rule_book = [("NN","MD","P1TP1T","NN NN")]

  # Create
  error_dict = defaultdict(lambda: defaultdict(int))

  # Create a copy of the starting sentence
  sentence = master_sentence

  # Determine the current error rate based on the starting sentence
  current_errors, error_count = count_mistagged(sentence,answers,error_dict)

  # Initialize the counter for how many are mistagged
  most_mistagged = float('-inf')

  # Determine which combination of tags currently has the most errors
  for key in current_errors.keys():
    for secondary_key in current_errors[key].keys():
      if current_errors[key][secondary_key] > most_mistagged:
        most_mistaken = key
        most_mistaken_answer = secondary_key

  # While the accuracy isn't improving
  while last_accuracy > current_accuracy:

    # Search for the optimal rule to add to address the errors and add it to the rule book
    rule = find_optimal_rule(master_sentence,most_mistaken,most_mistaken_answer,answers,rule_book)
    rule_book.append(rule)

    # Run the rules in the rule book on the sentence and check the accuracy
    error_dict = defaultdict(lambda: defaultdict(int))
    sentence = master_sentence
    processed_sentence = process_rules(sentence,rule_book)
    current_errors, error_count = count_mistagged(processed_sentence,answers,error_dict)
    error_rate = error_count/len(sentence)
    accuracy = 1-error_rate

    # Find the new most mistagged thing
    most_mistagged = float('-inf')
    for key in current_errors.keys():
      for secondary_key in current_errors[key].keys():
        if current_errors[key][secondary_key] > most_mistagged:
          most_mistaken = key
          most_mistaken_answer = secondary_key


    print("Average Accuracy Rate: ", accuracy)

    # Update the accuracy checker
    last_accuracy = current_accuracy
    current_accuracy = accuracy

  # Return the completed rule book
  return rule_book

def find_optimal_rule(sentence, old_tag, new_tag, answers, rule_book):
  # Initialize a dict to track which rule has the lowest error rate
  rule_error_dict = defaultdict(lambda: defaultdict(int))

  ## Rule Set Up ##
  # Rule templates are broken up into different lists based on what kind of parameter they need
  # Rules that require tags need to be fed through the loop that sets their parameter to something from the tag bank
  # Rules that require 2 tags need to pass through an additional loop for getting stuff from the tag bank
  # Rules the require a word need to loop through the options for words
  RULE_TEMPLATES_TAGS_TWO_TAGS = ["P1TF1T","P1TP1T","F1TF1T"]
  RULE_TEMPLATES_TAGS = ["P1T","F1T","P2T","F2T","1P2T","1F2T","1P3T","1F3T"] + RULE_TEMPLATES_TAGS_TWO_TAGS
  RULE_TEMPLATES_WORDS = []
  RULE_TEMPLATES = RULE_TEMPLATES_TAGS + RULE_TEMPLATES_WORDS
  TAG_LIST = ["CC","CD","DT","EX","FW","IN","JJ","JJR","JJS","LS","MD","NN","NNS","NNP","NNPS","PDT","POS","PRP","PRP$","RB","RBR","RBS","RP","SYM","TO","UH","VB","VBD","VBG","VBN","VBP","VBZ","WDT","WP","WP$","WRB"]
  ####

  for rule_type in RULE_TEMPLATES:
    if rule_type in RULE_TEMPLATES_TAGS:
      if rule_type in RULE_TEMPLATES_TAGS_TWO_TAGS:
        for tag in TAG_LIST:
          for tag_2 in TAG_LIST:
            cur_rule = (old_tag,new_tag,rule_type,tag + " " + tag_2)
            sentence = process_rules(sentence,rule_book)
            for word_idx in range(len(sentence)):
              processed_sentence = rule_parser(cur_rule,sentence,word_idx)
            current_errors, error_count = count_mistagged(processed_sentence,answers,defaultdict(lambda: defaultdict(int)))
            rule_error_dict[cur_rule] = error_count
      else:
        for tag in TAG_LIST:
          cur_rule = (old_tag,new_tag,rule_type,tag)
          processed_sentence = process_rules(sentence,rule_book)
          for word_idx in range(len(processed_sentence)):
            processed_sentence = rule_parser(cur_rule,sentence,word_idx)
          current_errors, error_count = count_mistagged(processed_sentence,answers,defaultdict(lambda: defaultdict(int)))
          rule_error_dict[cur_rule] = error_count


  most_improvement = float('inf')
  for key in rule_error_dict.keys():
    if rule_error_dict[key] < most_improvement:
      most_improvement = rule_error_dict[key]
      best_rule = key

  old_current_errors, old_error_count = count_mistagged(processed_sentence,answers,defaultdict(lambda: defaultdict(int)))
  if old_error_count == most_improvement:
    return (new_tag,new_tag,RULE_TEMPLATES_TAGS[0],new_tag)
  else:
    return best_rule

def process_rules(sentence,rule_book):
  processed_sentence = sentence
  for word_idx in range(len(processed_sentence)):
    for rule in rule_book:
      processed_sentence = rule_parser(rule,processed_sentence,word_idx)
  return processed_sentence

# Checks how many errors occur under the current rules
def count_mistagged(guesses,answers,error_dict):
  incorrect_tags = 0
  for guess_idx in range(len(guesses)):
    if guesses[guess_idx][1] != answers[guess_idx][1]:
      incorrect_tags += 1
      error_dict[answers[guess_idx][1]][guesses[guess_idx][1]] += 1
  return error_dict, incorrect_tags

# Takes a string list sentence and converts it to a list of tuples
def init_sentence(sentence):
  sentence_tuple = []
  for word in sentence:
    ## These initializing conditions improve the brill tagger's performance (https://dream-y.medium.com/parts-of-speech-tagging-and-brill-tagging-ee57a157dfe5)
    if word != word.lower():
      sentence_tuple.append((word,"NNP"))
    elif word[len(word)-3:len(word)] == "ous" or word[len(word)-2:len(word)] == "ly":
      sentence_tuple.append((word,"JJ"))
    elif word[len(word)-3:len(word)] == "ion":
      sentence_tuple.append((word,"NN"))
    else:
      sentence_tuple.append((word,"NN"))
  return sentence_tuple



answer_sentence = [("mark", "NN"), ("page", "NN"), ("will", "MD"), ("close", "VB"), ("the","JJ"), ("book", "NN")]
test_sentence = [word[0] for word in answer_sentence]

rule=("NN","VB","P1T","NN")
idx=2

initalized_sentence = init_sentence(test_sentence)
print(initalized_sentence)

#current_errors, error_count = count_mistagged(rule_parser(rule,initalized_sentence,idx),answer_sentence,error_dict)


final_rule_book = train(initalized_sentence, answer_sentence)
print("Final Rule Book:", final_rule_book)

processed_sentence = process_rules(initalized_sentence,[("NN","MD","P1TP1T","NN NN")])

print("Answer Sentence: ", answer_sentence)
print("Processed Sentence: ", processed_sentence)
current_errors, error_count = count_mistagged(rule_parser(("NN","MD","P1TP1T","NN NN"),initalized_sentence,idx),answer_sentence,defaultdict(lambda: defaultdict(int)))
print("Percent Error: ", error_count/len(answer_sentence))

[('mark', 'NN'), ('page', 'NN'), ('will', 'NN'), ('close', 'NN'), ('the', 'NN'), ('book', 'NN')]
Average Accuracy Rate:  0.5
Average Accuracy Rate:  0.16666666666666663
Average Accuracy Rate:  0.16666666666666663
Final Rule Book: [('NN', 'MD', 'P1TP1T', 'NN NN'), ('NN', 'NN', 'P1T', 'NN'), ('NN', 'MD', 'P1T', 'CC'), ('MD', 'MD', 'P1T', 'MD')]
Answer Sentence:  [('mark', 'NN'), ('page', 'NN'), ('will', 'MD'), ('close', 'VB'), ('the', 'JJ'), ('book', 'NN')]
Processed Sentence:  [('mark', 'MD'), ('page', 'MD'), ('will', 'MD'), ('close', 'MD'), ('the', 'MD'), ('book', 'MD')]
Percent Error:  0.8333333333333334
